<a href="https://colab.research.google.com/github/spencer18001/machine-learning-zoomcamp/blob/main/09/hw_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install tensorflow=2.17.1
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [ ]:
import tensorflow as tf
from tensorflow import keras